In [15]:
input = r"""x00: 1
x01: 0
x02: 1
x03: 1
x04: 0
y00: 1
y01: 1
y02: 1
y03: 1
y04: 1

ntg XOR fgs -> mjb
y02 OR x01 -> tnw
kwq OR kpj -> z05
x00 OR x03 -> fst
tgd XOR rvg -> z01
vdt OR tnw -> bfw
bfw AND frj -> z10
ffh OR nrd -> bqk
y00 AND y03 -> djm
y03 OR y00 -> psh
bqk OR frj -> z08
tnw OR fst -> frj
gnj AND tgd -> z11
bfw XOR mjb -> z00
x03 OR x00 -> vdt
gnj AND wpb -> z02
x04 AND y00 -> kjc
djm OR pbm -> qhw
nrd AND vdt -> hwm
kjc AND fst -> rvg
y04 OR y02 -> fgs
y01 AND x02 -> pbm
ntg OR kjc -> kwq
psh XOR fgs -> tgd
qhw XOR tgd -> z09
pbm OR djm -> kpj
x03 XOR y03 -> ffh
x00 XOR y04 -> ntg
bfw OR bqk -> z06
nrd XOR fgs -> wpb
frj XOR qhw -> z04
bqk OR frj -> z07
y03 OR x01 -> nrd
hwm AND bqk -> z03
tgd XOR rvg -> z12
tnw OR pbm -> gnj
"""

In [17]:
input = r"""x00: 1
x01: 1
x02: 0
x03: 0
x04: 0
x05: 1
x06: 0
x07: 1
x08: 1
x09: 0
x10: 1
x11: 0
x12: 0
x13: 0
x14: 1
x15: 0
x16: 1
x17: 1
x18: 0
x19: 1
x20: 0
x21: 0
x22: 1
x23: 1
x24: 1
x25: 1
x26: 0
x27: 1
x28: 1
x29: 0
x30: 1
x31: 0
x32: 0
x33: 1
x34: 1
x35: 0
x36: 1
x37: 1
x38: 1
x39: 1
x40: 1
x41: 1
x42: 1
x43: 1
x44: 1
y00: 1
y01: 0
y02: 1
y03: 1
y04: 0
y05: 0
y06: 1
y07: 1
y08: 0
y09: 1
y10: 1
y11: 1
y12: 1
y13: 0
y14: 1
y15: 1
y16: 1
y17: 0
y18: 1
y19: 0
y20: 0
y21: 1
y22: 0
y23: 1
y24: 0
y25: 1
y26: 0
y27: 1
y28: 0
y29: 0
y30: 1
y31: 1
y32: 0
y33: 1
y34: 0
y35: 0
y36: 1
y37: 0
y38: 1
y39: 0
y40: 0
y41: 0
y42: 1
y43: 0
y44: 1

y08 AND x08 -> pkh
grg AND twt -> bbk
vvt OR wwt -> vgs
x10 XOR y10 -> pmq
pmq XOR hkf -> z10
vmw OR bfb -> hkf
twp OR qbq -> kmj
qns AND mwj -> qhk
dqm XOR cqp -> z02
snr AND crb -> htp
jwv XOR dgj -> z09
sjf OR rwf -> wkq
y02 XOR x02 -> dqm
msw AND qqp -> rss
fgv XOR bhw -> z26
y03 AND x03 -> ftb
kmj XOR fnh -> z24
jhv XOR bkq -> wtt
x27 XOR y27 -> knf
y40 XOR x40 -> jhf
gmq OR nrs -> kkq
y03 XOR x03 -> ndn
x25 XOR y25 -> hfj
x33 AND y33 -> rvf
wkh AND pmn -> vrq
mgg OR hbf -> fhw
nrr OR rtk -> wqr
x00 AND y00 -> mtk
crb XOR snr -> z30
y37 XOR x37 -> qns
kgw OR wms -> snr
y09 XOR x09 -> jwv
mtk AND kvc -> jjp
x01 XOR y01 -> kvc
jgg AND bsn -> fpm
kvt OR ftb -> drd
x15 AND y15 -> tfs
x34 XOR y34 -> jgv
y44 XOR x44 -> vjg
x21 XOR y21 -> fqr
x36 XOR y36 -> rwh
y30 AND x30 -> kgr
sqg OR hfb -> rfq
x05 XOR y05 -> kbj
rjq OR skn -> jhv
y17 AND x17 -> bmp
x28 AND y28 -> jwb
x44 AND y44 -> vrj
wwn XOR gwm -> z23
y39 AND x39 -> gsr
wkq XOR tfv -> z15
x31 AND y31 -> sst
x08 XOR y08 -> twt
wnf AND rsk -> pjp
mmr AND qdc -> rwf
y22 XOR x22 -> jsr
pnj OR bcc -> hhn
fpm OR wtv -> kth
kkq XOR jsr -> z22
srn XOR kth -> z33
vrq OR mrt -> tkc
wwn AND gwm -> qbq
jhf XOR mdn -> z40
y13 XOR x13 -> tqf
kvc XOR mtk -> z01
stg XOR kdv -> z35
grg XOR twt -> z08
kkq AND jsr -> fjh
tmd AND shm -> rjm
nvq AND chk -> gnp
twb OR ptq -> qqp
x32 XOR y32 -> bsn
y39 XOR x39 -> bkq
mph OR vbp -> stg
x02 AND y02 -> rks
dtj AND hfj -> frh
y43 XOR x43 -> qmr
chk XOR nvq -> z28
x16 AND y16 -> wnf
y06 XOR x06 -> ghf
rhr OR ccq -> qjq
x38 AND y38 -> rjq
tsf AND dtb -> twb
x15 XOR y15 -> tfv
vds AND gdk -> skn
hhn XOR rwh -> z36
x40 AND y40 -> bjf
wtd OR cks -> nvq
jpk OR qhk -> vds
x14 XOR y14 -> qdc
y19 AND x19 -> ptq
fnh AND kmj -> gqj
pjp OR vtj -> smg
x11 XOR y11 -> tmd
bbk OR pkh -> dgj
bkq AND jhv -> z39
y07 AND x07 -> krp
hkf AND pmq -> crj
y22 AND x22 -> cdk
knb OR vkk -> mmr
vds XOR gdk -> z38
jhf AND mdn -> qkq
vrj OR gkc -> z45
x41 AND y41 -> mrt
jgv XOR dkh -> z34
y16 XOR x16 -> vtj
rcf OR gqj -> dtj
rvf OR bqr -> dkh
x25 AND y25 -> swn
dbv XOR vhs -> z31
y12 XOR x12 -> fgq
y20 AND x20 -> kcf
rss OR kcf -> mjj
hfj XOR dtj -> z25
x14 AND y14 -> sjf
fpd XOR smg -> z17
x21 AND y21 -> z21
x43 AND y43 -> prt
x31 XOR y31 -> vhs
grv OR tfs -> rsk
y33 XOR x33 -> srn
msw XOR qqp -> z20
gsr OR wtt -> mdn
x30 XOR y30 -> crb
y24 AND x24 -> rcf
frh OR swn -> bhw
htm AND ndn -> kvt
x06 AND y06 -> hbf
x35 XOR y35 -> kdv
srn AND kth -> bqr
dgj AND jwv -> bfb
bsn XOR jgg -> z32
rjm OR skj -> tff
gcf AND fhw -> mpv
vfb OR jjp -> cqp
x00 XOR y00 -> z00
crj OR fgc -> shm
tff XOR fgq -> z12
mwj XOR qns -> z37
vjg AND kmg -> gkc
x19 XOR y19 -> dtb
hvs XOR tsn -> z18
dtb XOR tsf -> z19
y05 AND x05 -> srp
kkh AND bsb -> kgw
rks OR trv -> htm
frn AND ghf -> mgg
qjq XOR kbj -> frn
knf XOR vgs -> z27
y09 AND x09 -> vmw
x36 AND y36 -> dbc
knf AND vgs -> wtd
y23 XOR x23 -> wwn
wnf XOR rsk -> z16
y29 XOR x29 -> kkh
vjg XOR kmg -> z44
htp OR kgr -> dbv
htm XOR ndn -> z03
rfq XOR tqf -> z13
mjj AND fqr -> nrs
gcf XOR fhw -> z07
qmr AND wqr -> frs
x32 AND y32 -> wtv
x07 XOR y07 -> gcf
gmf OR pvh -> tsf
mjj XOR fqr -> gmq
qqs OR dbc -> mwj
krp OR mpv -> grg
x34 AND y34 -> vbp
tff AND fgq -> hfb
drd AND wkb -> rhr
tqf AND rfq -> vkk
x13 AND y13 -> knb
bhw AND fgv -> vvt
hhn AND rwh -> qqs
kbj AND qjq -> jcf
y41 XOR x41 -> wkh
y01 AND x01 -> vfb
jgv AND dkh -> mph
gnp OR jwb -> bsb
cdk OR fjh -> gwm
prt OR frs -> kmg
x17 XOR y17 -> fpd
y42 AND x42 -> rtk
fpd AND smg -> tqm
dbv AND vhs -> gdw
y29 AND x29 -> wms
kkh XOR bsb -> z29
cqp AND dqm -> trv
hvs AND tsn -> gmf
wkq AND tfv -> grv
x04 AND y04 -> ccq
x20 XOR y20 -> msw
y26 XOR x26 -> fgv
mmr XOR qdc -> z14
ghf XOR frn -> z06
y26 AND x26 -> wwt
sst OR gdw -> jgg
pmn XOR wkh -> z41
y04 XOR x04 -> wkb
y28 XOR x28 -> chk
wqr XOR qmr -> z43
kdv AND stg -> bcc
y18 AND x18 -> pvh
mvk XOR tkc -> z42
x27 AND y27 -> cks
bjf OR qkq -> pmn
tkc AND mvk -> nrr
y38 XOR x38 -> gdk
y37 AND x37 -> jpk
x23 AND y23 -> twp
tqm OR bmp -> tsn
wkb XOR drd -> z04
x42 XOR y42 -> mvk
y35 AND x35 -> pnj
shm XOR tmd -> z11
y24 XOR x24 -> fnh
x11 AND y11 -> skj
x10 AND y10 -> fgc
srp OR jcf -> z05
y12 AND x12 -> sqg
y18 XOR x18 -> hvs"""

In [20]:
# part 1

import re
from enum import StrEnum

class Operator(StrEnum):
    AND = 'AND'
    OR = 'OR'
    XOR = 'XOR'

from typing import NamedTuple
Rule = NamedTuple('Rule', [('operator', Operator), ('input1', str), ('input2', str), ('output', str)])

regex = r"(don't\(\)|do\(\)|mul\(\d+,\d+\))"

assignRegex = r"(\w+): (\d)"
ruleRegex = r"(\w+) (AND|OR|XOR) (\w+) -> (\w+)"

registers: dict[str, int] = {}
rules: list[Rule] = []
for line in input.splitlines():
    if (line == ""):
        continue
    result = re.search(assignRegex, line) 
    if (result != None):
        registers[result.group(1)] = int(result.group(2))
        continue
    result = re.search(ruleRegex, line) 
    if (result != None):
        rules.append(Rule(input1 = result.group(1), operator = result.group(2), input2 = result.group(3), output = result.group(4)))
        continue
    print (f"Not matched: {line}")
    raise NotImplementedError

print (len(registers), len(rules))

while True:
    numAssigned = 0
    for rule in rules:
        if (rule.output in registers): continue
        if (rule.input1 not in registers): continue
        if (rule.input2 not in registers): continue

        numAssigned += 1
        if (rule.operator == Operator.AND):
            registers[rule.output] = registers[rule.input1] & registers[rule.input2]
        elif (rule.operator == Operator.OR):
            registers[rule.output] = registers[rule.input1] | registers[rule.input2]
        elif (rule.operator == Operator.XOR):
            registers[rule.output] = registers[rule.input1] ^ registers[rule.input2]
        else:
            print (rule.operator)
            raise NotImplementedError
    if (numAssigned == 0):
        break

z = 0
for registername, binaryvalue in registers.items():
    if (registername[0] != 'z'): continue
    registernum = int(registername[1:])
    z = z | (binaryvalue << registernum)

print (z)






90 222
56939028423824


In [30]:
# part 2

# Old, inefficient 6-gate approach
# x01 XOR y01 -> a01
# a01 XOR carry01 -> z01
# x01 OR y01 -> b01
# b01 AND carry01 -> d01
# x01 AND y01 -> e01
# d01 OR e01 -> carry02


# 46 z outputs, 45 x/y inputs

# 222 rules -> 2 for initial half-adder, + 44 * 5 per adder


# 0, 0, 0 -> a01 = 0, z01 = 0, carry02 = 0
# 0, 0, 1 -> a01 = 0, z01 = 1, carry02 = 0
# 1, 0, 0 -> a01 = 1, z01 = 1, carry02 = 0
# 1, 0, 1 -> a01 = 1, z01 = 0, carry02 = 1
# 1, 1, 1 -> a01 = 0, z01 = 1, carry02 = 1

# 2-bit adder
# x00 AND y00 -> carry01
# x00 XOR y00 -> z00

# Full adder: two XOR, two AND, one OR

# x01 XOR y01 -> a01
# a01 XOR carry01 -> z01
# a01 AND carry01 -> b01
# x01 AND y01 -> d01
# b01 OR d01 -> carry02

## z02 = carry02



# part 1

import re
from enum import StrEnum

class Operator(StrEnum):
    AND = 'AND'
    OR = 'OR'
    XOR = 'XOR'

from typing import NamedTuple
Rule = NamedTuple('Rule', [('operator', Operator), ('input1', str), ('input2', str), ('output', str)])

regex = r"(don't\(\)|do\(\)|mul\(\d+,\d+\))"

assignRegex = r"(\w+): (\d)"
ruleRegex = r"(\w+) (AND|OR|XOR) (\w+) -> (\w+)"

registers: dict[str, int] = {}
rules: list[Rule] = []
for line in input.splitlines():
    if (line == ""):
        continue
    result = re.search(assignRegex, line) 
    if (result != None):
        registers[result.group(1)] = int(result.group(2))
        continue
    result = re.search(ruleRegex, line) 
    if (result != None):
        rules.append(Rule(input1 = result.group(1), operator = result.group(2), input2 = result.group(3), output = result.group(4)))
        continue
    print (f"Not matched: {line}")
    raise NotImplementedError


adderRules: list[Rule] = []
ruleSequence = []
regNameMap = {}

def inputsAre(rule: Rule, input1, input2):
    mappedInput1 = input1
    mappedInput2 = input2
    if (input1[0] != 'x' and input1[0] != 'y'):
        mappedInput1 = regNameMap[input1]
    if (input2[0] != 'x' and input2[0] != 'y'):
        mappedInput2 = regNameMap[input2]
    if (rule.input1 == mappedInput1 and rule.input2 == mappedInput2): return True
    if (rule.input1 == mappedInput2 and rule.input2 == mappedInput1): return True
    return False

# find first half-adder
for rule in rules:
    if (inputsAre(rule, 'x00', 'y00')):
        if (rule.operator == 'AND'):
            regNameMap['carry01'] = rule.output
            ruleSequence.append((rule, 'carry01'))
            rules.remove(rule)
        elif (rule.operator == 'XOR'):
            if (rule.output != "z00"): raise NotImplementedError
            regNameMap['z00'] = rule.output
            ruleSequence.append((rule, 'z00'))
            rules.remove(rule)
        else: raise NotImplementedError

print (len(rules))

def findNextInstruction(rule, currentBit, operator, prefix1, prefix2, outputprefix):
    bitStr = str(currentBit).zfill(2)
    found = False
    for rule in rules:
        if (inputsAre(rule, f'{prefix1}{bitStr}', f'{prefix2}{bitStr}') and rule.operator == operator):
            myName = f'{outputprefix}{bitStr}'
            if (outputprefix == 'carry'):
                myName = f'carry{str(currentBit+1).zfill(2)}'
            regNameMap[myName] = rule.output
            ruleSequence.append((rule, myName))
            rules.remove(rule)
            found = True
            break
    if (not found): print (regNameMap, ruleSequence, bitStr, instruction); raise NotImplementedError


targetBit = 45
currentBit = 0
instruction = 0
while currentBit != targetBit:
    # 0: x01 XOR y01 -> a01
    # 1: x01 AND y01 -> d01
    # 2: a01 XOR carry01 -> z01
    # 3: a01 AND carry01 -> b01
    # 4: b01 OR d01 -> carry02

    ## z02 = carry02

    currentBit += 1
    if (instruction == 0):
        findNextInstruction(rule, currentBit, )
        for rule in rules:
            if (inputsAre(rule, f'x{bitStr}', f'y{bitStr}') and rule.operator == 'XOR'):
                myName = f'a{bitStr}'
                regNameMap[myName] = rule.output
                ruleSequence.append((rule, myName))
                rules.remove(rule)
                instruction += 1
                break
        if (instruction == 0): print (regNameMap, ruleSequence, bitStr, instruction); raise NotImplementedError
    if (instruction == 1):
        for rule in rules:
            if (inputsAre(rule, f'x{bitStr}', f'y{bitStr}') and rule.operator == 'AND'):
                myName = f'd{bitStr}'
                regNameMap[myName] = rule.output
                ruleSequence.append((rule, myName))
                rules.remove(rule)
                instruction += 1
                break
        if (instruction == 1): print (regNameMap, ruleSequence, bitStr, instruction); raise NotImplementedError
    if (instruction == 2):
        for rule in rules:
            if (inputsAre(rule, f'a{bitStr}', f'carry{bitStr}') and rule.operator == 'XOR'):
                myName = f'z{bitStr}'
                regNameMap[myName] = rule.output
                ruleSequence.append((rule, myName))
                rules.remove(rule)
                instruction += 1
                break
        if (instruction == 2): print (regNameMap, ruleSequence, bitStr, instruction); raise NotImplementedError
    if (instruction == 3):
        for rule in rules:
            if (inputsAre(rule, f'a{bitStr}', f'carry{bitStr}') and rule.operator == 'AND'):
                myName = f'b{bitStr}'
                regNameMap[myName] = rule.output
                ruleSequence.append((rule, myName))
                rules.remove(rule)
                instruction += 1
                break
        if (instruction == 3): print (regNameMap, ruleSequence, bitStr, instruction); raise NotImplementedError
    if (instruction == 4):
        for rule in rules:
            if (inputsAre(rule, f'b{bitStr}', f'd{bitStr}') and rule.operator == 'OR'):
                myName = f'carry{str(currentBit+1).zfill(2)}'
                regNameMap[myName] = rule.output
                ruleSequence.append((rule, myName))
                rules.remove(rule)
                instruction = 0
                break
        if (instruction == 4): print (regNameMap, ruleSequence, bitStr, instruction); raise NotImplementedError
        










220
{'carry01': 'mtk', 'z00': 'z00', 'a01': 'kvc', 'd01': 'vfb', 'z01': 'z01', 'b01': 'jjp', 'carry02': 'cqp', 'a02': 'dqm', 'd02': 'rks', 'z02': 'z02', 'b02': 'trv', 'carry03': 'htm', 'a03': 'ndn', 'd03': 'ftb', 'z03': 'z03', 'b03': 'kvt', 'carry04': 'drd', 'a04': 'wkb', 'd04': 'ccq', 'z04': 'z04', 'b04': 'rhr', 'carry05': 'qjq', 'a05': 'kbj', 'd05': 'srp', 'z05': 'frn', 'b05': 'jcf', 'carry06': 'z05', 'a06': 'ghf', 'd06': 'hbf'} [(Rule(operator='AND', input1='x00', input2='y00', output='mtk'), 'carry01'), (Rule(operator='XOR', input1='x00', input2='y00', output='z00'), 'z00'), (Rule(operator='XOR', input1='x01', input2='y01', output='kvc'), 'a01'), (Rule(operator='AND', input1='y01', input2='x01', output='vfb'), 'd01'), (Rule(operator='XOR', input1='kvc', input2='mtk', output='z01'), 'z01'), (Rule(operator='AND', input1='mtk', input2='kvc', output='jjp'), 'b01'), (Rule(operator='OR', input1='vfb', input2='jjp', output='cqp'), 'carry02'), (Rule(operator='XOR', input1='y02', input2='x0

NotImplementedError: 